In [293]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import get_perms

Lx, Ly = (4, 4)
L= Lx*Ly

# construct Hamiltonian
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

perms_v, perms_h = get_perms(Lx, Ly)
perms_extended = [[perms_v[0]]] + [perms_v]*3 + [[perms_v[0]], [perms_h[0]]] +\
                    [perms_h]*3 + [[perms_h[0]], [perms_v[0]]] + [perms_v]*3 + [[perms_v[0]]]
perms_ext_reduced = [perms_v]*3  + [perms_h]*3 + [perms_v]*3

In [294]:
import numpy as np


def left_normalize(Ms):
    As = []
    T = np.ones((1, 1))
    for M in Ms:
        M = np.tensordot(T, M, axes=(1, 1)) 
        M = np.transpose(M, [1, 0, 2])
        d, chi1, chi2 = M.shape             
        U, S, Vh = np.linalg.svd(np.reshape(M, [d*chi1, chi2]), full_matrices=False)
        A = np.reshape(U, [d, chi1, -1])   
        As.append(A)                        
        T = np.diag(S) @ Vh                 

    # Keep leftover signs (but no normalization)
    As[0] = As[0]*np.sign(T)
    return As

def right_normalize(Ms):
    Bs = []
    T = np.ones((1, 1))
    for M in Ms[::-1]:
        M = np.tensordot(M, T, axes=(2, 0))
        d, chi1, chi2 = M.shape
        M = np.reshape(M, [chi1, d*chi2])
        U, S, Vh = np.linalg.svd(M, full_matrices=False)
        _, chi_s = U.shape
        Bs.append(Vh.reshape([chi_s, d, chi2]).transpose([1, 0, 2]))
        T = U@np.diag(S)
    Bs[0] = Bs[0] * np.sign(T)
    return Bs[::-1]

    
def random_mps(L_plus_1, max_bond_dim=None, anc=None):
    D = max_bond_dim if max_bond_dim is not None else np.inf
    
    d = 2  # Qubit system (dim=2)
    mps = []

    D1 = min(D, d)
    A0 = np.zeros((d, 1, D1), dtype=np.complex128)
    if anc is None:
        A0[0, 0, :] = np.random.randn(D1) + 1j * np.random.randn(D1)  # only |0⟩ component
        A0[1, 0, :] = np.random.randn(D1) + 1j * np.random.randn(D1)  # only |0⟩ component
    elif anc==0:
        A0[0, 0, :] = np.random.randn(D1) + 1j * np.random.randn(D1)  # only |0⟩ component
    else:
        A0[1, 0, :] = np.random.randn(D1) + 1j * np.random.randn(D1)  # only |0⟩ component
        
    A0 /= np.linalg.norm(A0)  # normalize
    mps.append(A0)
        
    Dl = D1
    # Middle sites (1 to L-1)
    for i in range(1, L_plus_1 - 1):
        Dr = min(D, d * Dl)
        A = np.random.randn(d, Dl, Dr) + 1j * np.random.randn(d, Dl, Dr)
        A /= np.linalg.norm(A)
        mps.append(A)
        Dl = Dr

    # Last site (site L): (d=2, Dl, Dr=1)
    A_last = np.random.randn(d, Dl, 1) + 1j * np.random.randn(d, Dl, 1)
    A_last /= np.linalg.norm(A_last)
    mps.append(A_last)
    mps = left_normalize(mps)
    return mps


# Example usage:
L_plus_1 = 13
mps_tensors = random_mps(L_plus_1, max_bond_dim=10)

In [295]:
import numpy as np

def mps_fidelity(mps1, mps2):
    assert len(mps1) == len(mps2), "MPSs must have same length"

    # Start with scalar "environment" = 1
    env = np.ones((1, 1))  # shape (1, 1)
    for A, B in zip(mps1, mps2):
        env = np.tensordot(env, A, axes=(0, 1))
        env = np.tensordot(env, B.conj(), axes=([0, 1], [1, 0]))
    # env should now be scalar (1x1)
    inner_product = env[0, 0]
    fidelity = np.linalg.norm(inner_product) ** 2
    return fidelity.real


In [296]:
import numpy as np


def get_mps_of_sv(state_vector, max_bond_dim=None, cutoff=1e-10):
    sv = np.asarray(state_vector, dtype=complex)
    N = int(np.log2(sv.size))
    assert 2 ** N == sv.size, "Input size must be a power of 2 (qubit system)"
    
    mps = []
    psi = sv.reshape([2] * N)  # Turn into N-dimensional tensor

    # Left-to-right SVD sweep
    for n in range(N - 1):
        D = psi.shape[0]
        psi = psi.reshape(D, -1)  # Flatten remaining qubits
        # SVD
        U, S, Vh = np.linalg.svd(psi, full_matrices=False)

        # Truncate
        #keep = (S > cutoff)
        keep = np.where(S > cutoff)[0]
        if max_bond_dim is not None:
            keep = keep[:max_bond_dim]
        U = U[:, keep]
        S = S[keep]
        Vh = Vh[keep, :]

        Dl = U.shape[0] // 2
        Dr = U.shape[1]
        A = U.reshape(Dl, 2, Dr).transpose([1, 0, 2])
        mps.append(A)
        
        psi = np.diag(S) @ Vh  # Pass remainder to next step
        psi = psi.reshape(Dr * 2, -1)
        

    A_last = psi.reshape(psi.shape[0] // 2, 2, 1).transpose([1, 0, 2])
    mps.append(A_last)
    return mps


def mps_to_state_vector(mps):
    N = len(mps)
    psi = mps[0]  # shape: (2, 1, D1)

    # Contract left to right
    for i in range(1, N):
        A = mps[i]
        psi = np.tensordot(psi, A, axes=(psi.ndim-1, 1))
        psi = psi.transpose([i for i in range(psi.ndim-3)] + [psi.ndim-2, psi.ndim-3] + [psi.ndim-1])

    psi = np.squeeze(psi)  # removes axes of length 1
    psi = psi.reshape(-1)
    return psi


mps_random = random_mps(16, max_bond_dim=2**8)
sv = mps_to_state_vector(mps_random)
mps_back = get_mps_of_sv(sv, max_bond_dim=2**8)

#print("Original:", sv)
#print("Recovered:", sv_back)
print("Fidelity:", mps_fidelity(mps_random, mps_back))  # Should be ≈ 1

Fidelity: 1.0000000000000058


In [297]:
sv = random_statevector(2**10)
mps_back = get_mps_of_sv(sv)
sv_back = mps_to_state_vector(mps_back)

print("Fidelity:", np.linalg.norm(np.vdot(sv_back, sv))**2) # Should be ≈ 1

Fidelity: 1.0000000000000018


In [298]:
import numpy as np

X = np.array([[0, 1], [1, 0]])

def apply_two_site_operator(mps, gate, site, max_bond_dim=None, cutoff=1e-10):
    A1 = mps[site]      # shape (2, Dl, Dmid)
    A2 = mps[site + 1]  # shape (2, Dmid, Dr)

    d, Dl, Dmid = A1.shape
    d2, Dmid2, Dr = A2.shape
    assert d == d2 == 2 and Dmid == Dmid2

    # Combine A1 and A2 into one tensor
    theta = np.tensordot(A1, A2, axes=(2, 1))  # (2, Dl, Dmid) x (2, Dmid, Dr) → (2, Dl, 2, Dr)
    theta = np.transpose(theta, (0, 2, 1, 3))  # → (2,2,Dl,Dr)
    theta = theta.reshape(4, Dl * Dr)          # → (4, Dl*Dr)

    # Apply gate
    theta = np.tensordot(gate, theta, axes=(1, 0))  # (4,4) x (4, Dl*Dr)
    theta = theta.reshape(2, 2, Dl, Dr)        # back to (2,2,Dl,Dr)
    theta = np.transpose(theta, (0, 2, 1, 3))  # → (2,Dl,2,Dr)
    theta = theta.reshape(2 * Dl, 2 * Dr)      # → (2*Dl, 2*Dr)

    # SVD
    U, S, Vh = np.linalg.svd(theta, full_matrices=False)
    
    # Truncate
    keep = np.where(S > cutoff)[0]
    if max_bond_dim is not None:
        keep = keep[:max_bond_dim]
    U = U[:, keep]
    S = S[keep]
    Vh = Vh[keep, :]

    # Rebuild A1 and A2
    new_D = len(S)
    U = U.reshape(2, Dl, new_D)
    Vh = (np.diag(S)@Vh).reshape(new_D, 2, Dr).transpose([1, 0, 2])

    mps[site] = U
    mps[site + 1] = Vh

    return mps


def apply_localGate(mps, op, k, l, max_bond_dim=None):
    SWAP = [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]
    if k == l:
        raise ValueError("Sites k and l must be different")
    if k > l:
        k, l = l, k
        op = SWAP @ op @ SWAP
    
    for i in range(l-1, k, -1):
        mps = apply_two_site_operator(mps, SWAP, i)
    
    # Now sites k and k+1 correspond to original sites k and l
    mps = apply_two_site_operator(mps, op, k, max_bond_dim=max_bond_dim)
    
    for i in range(k+1, l):
        mps = apply_two_site_operator(mps, SWAP, i)
    
    return mps


sv = random_statevector(16).data
sv_new1 = np.kron(np.kron(np.kron(X, I2), I2), X) @ sv
mps_new1 = get_mps_of_sv(sv_new1)

mps = get_mps_of_sv(sv)
mps_new2 = apply_localGate(mps, np.kron(X, X), 0, 3)

mps_fidelity(mps_new1, mps_new2)

1.0000000000000009

In [299]:
mps = random_mps(2)
sv = mps_to_state_vector(mps) 
mps_1 = bring_sites_together_and_apply(mps, np.kron(X, X), 0, 1)
mps_2 = get_mps_of_sv(np.kron(X, X) @ sv)

mps_fidelity(mps_1, mps_2)

0.9999999999999993

In [305]:
import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms, Z
import rqcopt as oc


def trotter(mps, t, L, Lx, Ly, J, g, dag=False, max_bond_dim=None):
    nsteps = int(np.ceil(t/0.1))
    t = t/nsteps
    perms_v, perms_h = get_perms(Lx, Ly)
    indices = oc.SplittingMethod.suzuki(2, 1).indices
    coeffs = oc.SplittingMethod.suzuki(2, 1).coeffs
    
    hloc1 = J*np.kron(Z, Z)
    hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/4
    hlocs = (hloc1, hloc2)
    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))

    for n in range(nsteps):
        for layer, V in enumerate(Vlist_start):
            for perm in perms_v:
                for j in range(len(perm)//2):
                    mps = apply_localGate(mps, V, perm[2*j]+1, perm[2*j+1]+1, max_bond_dim=max_bond_dim)
                
            for perm in perms_h:
                for j in range(len(perm)//2):
                    mps = apply_localGate(mps, V, perm[2*j]+1, perm[2*j+1]+1, max_bond_dim=max_bond_dim)

    return mps

    
for t in [.25]:
    mps = random_mps(L, max_bond_dim=2**2)
    state = mps_to_state_vector(mps)

    A0 = np.zeros((2, 1, 1), dtype=np.complex128)
    A0[1, :, :] = 1
    mps = [A0]+mps
    
    mps = trotter(mps, t, L, Lx, Ly, J, g, max_bond_dim=2**8)
    
    exact_v1 = np.kron(np.array([0, 1]), expm_multiply(-1j * t * hamil, state))
    mps_exact = get_mps_of_sv(exact_v1)
    print(mps_fidelity(mps_exact, mps))


0.9972606427520847


In [312]:
Vlist = []
with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_rS1_niter15_3hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]
    
control_layers = [0, 4, 5, 9, 10, 14]
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_niter20_rS5_DECOMPOSE_n9_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]


def ccU(mps, Vlist, Xlists_opt, perms, perms_qc, control_layers, max_bond_dim=None):
    mps = apply_localGate(mps, np.kron(X, I2), 0, 1, max_bond_dim=max_bond_dim)
    for i, V in enumerate(Vlist):
        layer = i
        if i in control_layers:
            for perm in perms[layer]:
                for j in range(L//2):
                    mapp = {0: 0, 1: perm[2*j]+1, 2:perm[2*j+1]+1}
                    for l, G in enumerate(Xlists_opt[i]):
                        mps = apply_localGate(mps, G, mapp[perms_qc[l][0]], mapp[perms_qc[l][1]], max_bond_dim=max_bond_dim)
        else:
            for perm in perms[layer]:
                for j in range(len(perm)//2):
                    mps = apply_localGate(mps, V, perm[2*j]+1, perm[2*j+1]+1, max_bond_dim=max_bond_dim)
    mps = apply_localGate(mps, np.kron(X, I2), 0, 1, max_bond_dim=max_bond_dim)

    return mps


t = 0.25
mps = random_mps(L, max_bond_dim=2**2)
state = mps_to_state_vector(mps)

A0 = np.zeros((2, 1, 1), dtype=np.complex128)
A0[0, :, :] = 1
mps = [A0]+mps
mps = ccU(mps, Vlist, Xlists_opt, perms_extended, perms_qc, control_layers, max_bond_dim=2**8)

exact_v1 = np.kron(np.array([1, 0]), expm_multiply(1j * t * hamil, state))
mps_exact = get_mps_of_sv(exact_v1)
print(mps_fidelity(mps_exact, mps))

0.9996714843864107


In [313]:
t = 0.25
mps = random_mps(L, max_bond_dim=2**2)
state = mps_to_state_vector(mps)

A0 = np.zeros((2, 1, 1), dtype=np.complex128)
A0[1, :, :] = 1
mps = [A0]+mps
mps = ccU(mps, Vlist, Xlists_opt, perms_extended, perms_qc, control_layers, max_bond_dim=2**8)

exact_v2 = np.kron(np.array([0, 1]), expm_multiply(-1j * t * hamil, state))
mps_exact = get_mps_of_sv(exact_v2)
print(mps_fidelity(mps_exact, mps))

0.9996462570417276


In [ ]:
"""perms_v, perms_h = (
[[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
 [1, 2, 3, 4, 5, 0, 7, 8, 9, 10, 11, 6, 13, 14, 15, 16, 17, 12, 19, 20, 21, 22, 23, 18, 25, 26, 27, 28, 29, 24, 31, 32, 33, 34, 35, 30]],
[[0, 6, 12, 18, 24, 30, 1, 7, 13, 19, 25, 31, 2, 8, 14, 20, 26, 32, 3, 9, 15, 21, 27, 33, 4, 10, 16, 22, 28, 34, 5, 11, 17, 23, 29, 35], 
 [6, 12, 18, 24, 30, 0, 7, 13, 19, 25, 31, 1, 8, 14, 20, 26, 32, 2, 9, 15, 21, 27, 33, 3, 10, 16, 22, 28, 34, 4, 11, 17, 23, 29, 35, 5]]
 )
"""